# Science Case #1:

Find all <source-type> within a given area of the sky with emission from
<molecule(s)> detected.


The science motivation might be to find all young stellar objects (YSOs) in Taurus with ALMA
detections of CO J=2-1. The return from this query would include images of the moment maps of
CO emission, peak intensity, resolution, noise, correlator setting, and other relevant information
from any Taurus YSOs (as identified by science keywords, abstract text, SIMBAD coordinate
matches) in the publicly accessible ALMA Science Archive. The user could identify which data
are appropriate for their study an pull the u,v data or image cubes of interest from the archive.

Optionally, the user could then create a query to retrieve the full set of science products for all
sources, or the sources of interest. This would allow exploration of continuum and other lines
detected in the same observations. The quantitative data are returned as tables in the user’s
Python environment (for example Jupyter notebook) which can then be manipulated (see Section
4), for example to produce a plot of continuum flux versus CO integrated intensity or CO
intensity versus [13CO/12CO] intensity ratio.



In [ ]:
from astroquery.admit import ADMIT
from astroquery.alma import Alma, tapsql
from astropy.coordinates import SkyCoord
import pandas as pd
# display the whole table inthe notebook
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth',25)


In [ ]:
result1 = Alma.query_tap("select * from ivoa.obscore WHERE target_name = 'NGC3504'")
result2 = Alma.query_tap("select * from ivoa.obscore WHERE target_name = 'ngc3504'")
coord = SkyCoord.from_name('ngc3504')
print(coord)
sqlcoord = tapsql._gen_pos_sql('SkyCoord.from_name','ngc3504')
print(sqlcoord)
sql = "select * from ivoa.obscore WHERE source_name_resolver = '" +sqlcoord+"'"
print(sql)
if False:
    result3 = Alma.query_tap(sql) # fails         

In [ ]:
a = ADMIT()
a.check()
a.key_description

In [ ]:
a.keys

In [ ]:
result = a.query(formula="CO", transition="2-1")
len(result)

In [ ]:
result

In [ ]:
result=a.query(source_name_alma="NGC3049",mom0flux=">0")
len(result) 

In [ ]:
result

In [ ]:
sql = "select * from alma inner join win on (win.a_id = alma.id) \
inner join sources on (sources.w_id = win.id) inner join lines on (lines.w_id = win.id )\
where  target_name='NGC3049'"

In [ ]:
result = pd.DataFrame(a.sql(sql))
len(result)

In [ ]:
result

In [ ]:
sql = "select * from alma inner join win on (win.a_id = alma.id) \
inner join sources on (sources.w_id = win.id) \
where  target_name='NGC3049'"
result =  pd.DataFrame(a.sql(sql))
len(result)

In [ ]:
result

In [ ]:
sql = "select * from alma inner join win on (win.a_id = alma.id) \
inner join sources on (sources.w_id = win.id) inner join lines on (lines.w_id = win.id )\
where  target_name='NGC3049' and formula like '%CO%'"
result =  pd.DataFrame(a.sql(sql))
len(result)

In [ ]:
result=a.query(source_name_alma='NGC3049',formula="*CO*")
len(result)

In [ ]:
result

In [ ]:
result=a.query(source_name_alma='NGC3504',formula="CS")
len(result)

In [ ]:
result

In [ ]:
result1 = a.query(source_name_alma="NGC3504")
#result2 = a.query(source_name_resolver='ngc3504')
sql = "select * from alma where alma.target_name='NGC3504'"
result2 = a.sql(sql)
result2=pd.DataFrame(result2,columns=a._colnames['alma'])
result3 = a.query(source_name_alma="NGC3504",mom0flux=">-1")

In [ ]:
print(len(result1),len(result2),len(result3))
pd.set_option('display.max_colwidth',-1)

In [ ]:
result3['obs_id']

In [ ]:
result=a.query(scientific_category="galaxy evolution")  # case sensitive! returns zero results
len(result)

In [ ]:
result=a.query(win_snr=">1",source_name_alma="NGC3049") 
len(result) #hmmm zero. 

-------------------
tinker code below

In [ ]:
_gen_numeric_sql = _gen_str_sql = _gen_pos_sql = _gen_spec_res_sql = _gen_band_list_sql = _gen_datetime_sql = _gen_pol_sql = None

In [ ]:
ADMIT_FORM_KEYS = {
    'Window': {  
        'ALMA ID': ['alma_id','win.alma_id',_gen_numeric_sql],
        'Spectral Window': [ 'spw','win.id', _gen_numeric_sql],
        'Number of Lines': [ 'nlines','win.nlines', _gen_numeric_sql],
        'Number of Sources': [ 'nsources','win.nsources', _gen_numeric_sql],
        'Number of Channels': [ 'nchan','win.nchan', _gen_numeric_sql],
        'RMS noise': [ 'rms','win.rms', _gen_numeric_sql],
     },
    'Lines': {
        'Spectral Window': [ 'spw','lines.w_id', _gen_numeric_sql],
        'Formula': [ 'formula','lines.formula', _gen_str_sql],
        'Transition': [ 'transition','lines.transition', _gen_str_sql],
        'Velocity': [ 'velocity','lines.velocity', _gen_numeric_sql],
        # we are not using this
        #'Channels': [ 'chan','lines.chan', _gen_numeric_sql],
     },
    'Sources': {
        'Spectral Window': [ 'spw','sources.w_id', _gen_numeric_sql],
        'Line ID': [ 'lines_id','sources.lines_id', _gen_numeric_sql],
        'RA (Degrees)': ['ra', 'sources.ra',  _gen_numeric_sql],
        'Dec (Degrees)': ['dec', 'sources.dec',  _gen_numeric_sql],
        'Flux': ['flux', 'sources.flux',  _gen_numeric_sql],
        # source.snr is not actually a table column but we can use it as 
        # a trigger to munge some sql post-facto.
        'Signal to Noise Ratio': ['snr', 'sources.snr',  _gen_numeric_sql],
     },
    'Header': { # no science use case
        'Key': ['header_key','header.key',_gen_str_sql],
        'Value': ['header_val','header.val',_gen_str_sql],
     },
    'Cont': {
        'ALMA ID': ['alma_id','cont.alma_id',_gen_numeric_sql],
        'Bands': ['band','cont.cont',_gen_str_sql],
        'Number of Continuum Sources': ['nc_sources','cont.nsources',_gen_numeric_sql],
     },
    'Alma': {
        'Observation': ['obs_id','alma.obs_id',_gen_str_sql],
        # From here below are just a copy of ALMA_FORM_KEYS without the external wrapper dict.
        # Position
        'Source name (astropy Resolver)': ['source_name_resolver',
                                           'SkyCoord.from_name', _gen_pos_sql],
        'Source name (ALMA)': ['source_name_alma', 'alma.target_name', _gen_str_sql],
        'RA Dec (Sexagesimal)': ['ra_dec', 'alma.s_ra, alma.s_dec', _gen_pos_sql],
        'Galactic (Degrees)': ['galactic', 'alma.gal_longitude, alma.gal_latitude',
                               _gen_pos_sql],
        'Angular resolution (arcsec)': ['spatial_resolution',
                                        'alma.spatial_resolution', _gen_numeric_sql],
        'Largest angular scale (arcsec)': ['spatial_scale_max',
                                           'alma.spatial_scale_max', _gen_numeric_sql],
        'Field of view (arcsec)': ['fov', 'alma.s_fov', _gen_numeric_sql],
        # Energy
        'Frequency (GHz)': ['frequency', 'alma.frequency', _gen_numeric_sql],
        'Bandwidth (Hz)': ['bandwidth', 'alma.bandwidth', _gen_numeric_sql],
        'Spectral resolution (KHz)': ['spectral_resolution',
                                      'alma.em_resolution', _gen_spec_res_sql],
        'Bandesult = a.query(formula="CO", transition="2-1")': ['band_list', 'alma.band_list', _gen_band_list_sql],
        # Time
        'Observation date': ['start_date', 'alma.t_min', _gen_datetime_sql],
        'Integration time (s)': ['integration_time', 'alma.t_exptime',
                                 _gen_numeric_sql],
        # Polarization
        'Polarisation type (Single, Dual, Full)': ['polarisation_type',
                                                   'alma.pol_states', _gen_pol_sql],
        # Observation
        'Line sensitivity (10 km/s) (mJy/beam)': ['line_sensitivity',
                                                  'alma.sensitivity_10kms',
                                                  _gen_numeric_sql],
        'Continuum sensitivity (mJy/beam)': ['continuum_sensitivity',
                                             'alma.cont_sensitivity_bandwidth',
                                             _gen_numeric_sql],
        'Water vapour (mm)': ['water_vapour', 'alma.pvw', _gen_numeric_sql],
        # Project
        'Project code': ['project_code', 'alma.proposal_id', _gen_str_sql],
        'Project title': ['project_title', 'alma.obs_title', _gen_str_sql],
        'PI name': ['pi_name', 'alma.obs_creator_name', _gen_str_sql],
        'Proposal authors': ['proposal_authors', 'alma.proposal_authors', _gen_str_sql],
        'Project abstract': ['project_abstract', 'alma.proposal_abstract', _gen_str_sql],
        'Publication count': ['publication_count', 'alma.NA', _gen_str_sql],
        'Science keyword': ['science_keyword', 'alma.science_keyword', _gen_str_sql],
       # Publication'
        'Bibcode': ['bibcode', 'alma.bib_reference', _gen_str_sql],
        'Title': ['pub_title', 'alma.pub_title', _gen_str_sql],
        'First author': ['first_author', 'alma.first_author', _gen_str_sql],
        'Authors': ['authors', 'alma.authors', _gen_str_sql],
        # this may need special handling? or person does search on pub_abstract="*YSO*" 
        'Abstract': ['pub_abstract', 'alma.pub_abstract', _gen_str_sql],
        'Year': ['publication_year', 'alma.pub_year', _gen_numeric_sql],
     },
}

In [ ]:
s = ['formula', 'lines.formula', None]
s in ADMIT_FORM_KEYS["Lines"].values()

In [ ]:
ADMIT_FORM_KEYS["Lines"].values()

In [ ]:
2.lower()